In [2]:
# Colab 셀에서 실행
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-42c20tia/unsloth_e8e337aefdd64dfb82bfb358be78636b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-42c20tia/unsloth_e8e337aefdd64dfb82bfb358be78636b
  Resolved https://github.com/unslothai/unsloth.git to commit b2143c6b61221bf7717311f640f2cdf51ecefa8b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.6/180.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.5 MB/s eta 0:00:00


In [3]:
from unsloth import FastLanguageModel
import torch

# ---------------------------------------------------------
# ✅ 사용자가 요청한 모델 ID 설정
model_name = "Qwen/Qwen3-0.6B"
# ---------------------------------------------------------

max_seq_length = 2048 # 문맥 길이
dtype = None # None으로 두면 자동 설정 (Float16/Bfloat16)
load_in_4bit = True # 4bit 양자화로 메모리 절약 (무료 GPU에서 필수)

# 모델 및 토크나이저 불러오기
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        # token = "hf_..." # 비공개 모델일 경우 여기에 토큰 입력 필요
    )
    print(f"✅ {model_name} 모델 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("팁: 해당 모델이 Hugging Face에 실제로 존재하는지, 혹은 오타가 없는지 확인해주세요.")
    print("대체 추천 모델: 'Qwen/Qwen2.5-0.5B-Instruct'")

# LoRA(Low-Rank Adaptation) 설정
# 작은 모델이라도 파라미터 전체를 학습하면 과적합되므로 LoRA 사용 권장
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank (성능과 메모리 밸런스, 16~64 추천)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ Qwen/Qwen3-0.6B 모델 로드 성공!


Unsloth 2025.12.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
import json
from datasets import Dataset

# 파일 경로
file_path = "/content/data.json"

# 데이터 로드
data = []
try:
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue
    print(f"✅ 데이터 {len(data)}개 로드 완료.")
except FileNotFoundError:
    print("❌ '/content/data.json' 파일을 찾을 수 없습니다. 파일을 업로드해주세요.")

# 1. 시스템 프롬프트 별도 정의 (요청하신 내용 반영)
SYSTEM_PROMPT = (
    "You are an expert Korean translator.\n"
    "You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.\n"
    "Do not translate word-for-word; preserve meaning and tone in Korean."
)

# 2. 프롬프트 포맷 정의 (Alpaca 스타일 구조에 {system_prompt} 추가)
alpaca_prompt = """{system_prompt}

### Instruction:
{instruction}

### Input:
{input}

### Output:
{output}"""

# tokenizer는 이 코드 실행 이전에 정의되어 있어야 합니다.
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        # 3. 포맷팅 시 SYSTEM_PROMPT를 함께 주입
        text = alpaca_prompt.format(
            system_prompt = SYSTEM_PROMPT,  # 여기서 고정된 시스템 프롬프트 삽입
            instruction   = instruction,
            input         = input,
            output        = output
        ) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 데이터셋 생성
if data:
    dataset = Dataset.from_list(data)
    dataset = dataset.map(formatting_prompts_func, batched = True)

    # 확인용 출력
    print("\n[포맷팅된 데이터 예시]:")
    print(dataset[0]['text'])

✅ 데이터 600개 로드 완료.


Map:   0%|          | 0/600 [00:00<?, ? examples/s]


[포맷팅된 데이터 예시]:
You are an expert Korean translator.
You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.
Do not translate word-for-word; preserve meaning and tone in Korean.

### Instruction:
Don't translate it in Korean, but translate it according to Korean culture

### Input:
I actually saw him at the mall yesterday, no cap.

### Output:
나 어제 쇼핑몰에서 걔 진짜로 봤다니까, 구라 아님.<|im_end|>


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 데이터가 적으므로 60 step 정도면 충분 (약 3~4 epoch 효과)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

trainer_stats = trainer.train()
print("🎉 학습 완료!")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/600 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)


Step,Training Loss
1,3.927600
2,4.080600
3,3.826900
4,3.734800
5,3.544000
6,3.078400
7,2.745100
8,2.434900
9,2.016600
10,1.836000


🎉 학습 완료!


In [8]:
# 저장할 폴더 이름 (DPO 때 이 경로를 사용합니다)
save_path = "lora_sft_output"

# 1. LoRA 어댑터(학습된 가중치) 저장
model.save_pretrained(save_path)

# 2. 토크나이저 저장 (나중에 불러올 때 필수)
tokenizer.save_pretrained(save_path)

print(f"💾 모델과 토크나이저가 '{save_path}' 폴더에 저장되었습니다.")

💾 모델과 토크나이저가 'lora_sft_output' 폴더에 저장되었습니다.


In [10]:
import torch
import pandas as pd
import json
from unsloth import FastLanguageModel

# ------------------------------------------------------------------
# 1. 설정 (DPO 테스트와 동일한 시드 사용)
# ------------------------------------------------------------------
SEED = 42
model_path = "lora_sft_output"        # 👈 SFT 모델 경로로 변경됨
output_json_path = "sft_test_results.json" # 👈 저장할 JSON 파일명 변경
file_path = "test_data.csv"

# ------------------------------------------------------------------
# 2. SFT 모델 로드
# ------------------------------------------------------------------
print(f"🔄 SFT 모델 로드 중: {model_path} ...")
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_path,
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model)
    print("✅ 모델 로드 완료!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    raise e

# ------------------------------------------------------------------
# 3. 데이터 로드 및 샘플링 (동일한 시드 적용)
# ------------------------------------------------------------------
try:
    df = pd.read_csv(file_path)
    test_df = df.rename(columns={
        "영어 원문 (Source Text)": "input",
        "초월 번역": "output"
    })

    # DPO 때와 동일한 데이터셋 구성을 위해 같은 시드로 샘플링
    sample_size = 25
    real_size = min(len(test_df), sample_size)
    sampled_df = test_df.sample(n=real_size, random_state=SEED)

    print(f"📊 테스트 데이터: {len(sampled_df)}개 (Seed={SEED}) 로드 완료.\n")

except FileNotFoundError:
    print(f"❌ '{file_path}' 파일을 찾을 수 없습니다.")
    sampled_df = pd.DataFrame()

# ------------------------------------------------------------------
# 4. 프롬프트 및 생성 함수 (SFT도 동일한 프롬프트 사용 가정)
# ------------------------------------------------------------------
SYSTEM_PROMPT = (
    "You are an expert Korean translator.\n"
    "You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.\n"
    "Do not translate word-for-word; preserve meaning and tone in Korean."
)
INSTRUCTION = "Don't translate it in Korean, but translate it according to Korean culture"

def generate_translation(text):
    prompt = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n"
        f"Instruction: {INSTRUCTION}\n"
        f"Input: {text}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        if "assistant\n" in decoded:
            result = decoded.split("assistant\n")[-1].strip()
        else:
            result = decoded
    except:
        result = decoded

    return result

# ------------------------------------------------------------------
# 5. 실행 및 JSON 저장
# ------------------------------------------------------------------
results_list = []

if not sampled_df.empty:
    print(f"{'='*20} [SFT 모델 테스트 시작] {'='*20}\n")

    for idx, row in sampled_df.iterrows():
        input_text = row['input']
        ref_text = row['output']

        # 모델 예측
        model_out = generate_translation(input_text)

        # 콘솔 출력 (진행 상황 확인용)
        print(f"[Sample {idx}]")
        print(f"Ref : {ref_text}")
        print(f"SFT : {model_out}")
        print("-" * 40)

        # 결과 저장 구조 생성
        result_data = {
            "id": idx,
            "input": input_text,
            "reference": ref_text,
            "prediction_sft": model_out # DPO와 구분하기 위해 키 이름 변경 가능
        }
        results_list.append(result_data)

    # JSON 파일 저장
    try:
        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(results_list, f, ensure_ascii=False, indent=4)
        print(f"\n💾 SFT 결과 저장 완료: {output_json_path}")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")

else:
    print("⚠️ 데이터가 없습니다.")

🔄 SFT 모델 로드 중: lora_sft_output ...
==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ 모델 로드 완료!
📊 테스트 데이터: 25개 (Seed=42) 로드 완료.

==================== [SFT 모델 테스트 시작] ====================

[Sample 62]
Ref : 물들어 올때 노저어야 한다. 
SFT : <think>
어리게 해라, 중국인 이야기(인물 관계)는 나름이 어리네. 진짜 나도 잘 모르겠네.
----------------------------------------
[Sample 40]
Ref : 다 긴장했는데 걔 혼자 태평하더라
SFT : <think>
내가 말해보지 않지만, 한국어 문화에 맞춰 말해야 한다.
----------------------------------------
[Sample 95]
Ref : 그 남자는 주말 내내 소파에서 움직이질 않아.
SFT : 
----------------------------------------
[Sample 18]
Ref : 문제의 복잡성을 고려할 때, 해결책이 거저 주어질 것으로 기대되지 않으며,

In [11]:
pip install sacrebleu pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.2 MB/s eta 0:00:00


In [12]:
# 1. Unsloth의 DPO 패치 적용 (가장 윗줄에 있어야 함)
from unsloth import PatchDPOTrainer
PatchDPOTrainer()

import torch
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import DPOTrainer, DPOConfig
import json

# ---------------------------------------------------------
# ✅ 중요: SFT 학습이 완료된 모델 경로를 지정하세요.
# 예: SFT 코드가 model.save_pretrained("lora_sft")로 저장했다면 "lora_sft"
model_name = "lora_sft_output"
# (만약 SFT 직후 같은 세션에서 계속한다면 이 로드 과정은 생략 가능)
# ---------------------------------------------------------

max_seq_length = 2048
dtype = None
load_in_4bit = True

# 2. 모델 및 토크나이저 로드 (SFT된 가중치 로드)
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    print(f"✅ SFT 모델 로드 성공: {model_name}")
except:
    print("⚠️ 저장된 SFT 모델을 찾을 수 없어 기본 모델로 로드합니다. (실제 실험 시에는 SFT 모델 경로를 확인하세요)")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Qwen/Qwen2.5-0.5B-Instruct", # Qwen3가 없다면 2.5 사용
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

# 3. DPO를 위한 LoRA 설정 (SFT 모델 위에 덧씌우거나, 이어서 학습)
# Unsloth에서는 모델을 로드한 후 바로 DPO를 돌릴 수 있도록 설정해줍니다.
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# 4. 데이터 로드 및 포맷팅 (DPO 형식에 맞게 변환)
file_path_dpo = "data_dpo.json" # 업로드한 DPO 데이터 파일

# 시스템 프롬프트 (SFT와 동일하게 유지)
SYSTEM_PROMPT = (
    "You are an expert Korean translator.\n"
    "You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.\n"
    "Do not translate word-for-word; preserve meaning and tone in Korean."
)

EOS_TOKEN = tokenizer.eos_token # 문장 끝 토큰

def format_dpo_func(example):
    # 1) Prompt: 시스템 프롬프트 + 유저 입력 (Instruction + Input)
    # Assistant의 답변이 시작되기 직전까지만 작성합니다.
    prompt = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n"
        f"Instruction: {example['instruction']}\n"
        f"Input: {example['input']}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    # 2) Chosen / Rejected: 답변 내용 + EOS 토큰
    # DPO는 답변 부분만 학습하므로 여기에 전체 프롬프트를 다시 넣지 않습니다.
    return {
        "prompt": prompt,
        "chosen": example['chosen'] + EOS_TOKEN,
        "rejected": example['rejected'] + EOS_TOKEN,
    }

import json
from datasets import Dataset

# 파일 경로
file_path_dpo = "data_dpo.json"

# 데이터 로드 방식 수정 (일반 JSON 호환)
data_dpo = []
try:
    with open(file_path_dpo, "r", encoding="utf-8") as f:
        # 파일 전체를 한번에 읽어들입니다.
        # 만약 파일 내용이 [ ... ] 형태의 리스트라면 바로 로드됩니다.
        content = json.load(f)

        if isinstance(content, list):
            data_dpo = content
        elif isinstance(content, dict):
            # 만약 데이터가 하나만 있는 딕셔너리라면 리스트로 감싸줍니다.
            data_dpo = [content]
        else:
            print("⚠️ 데이터 형식이 올바르지 않습니다 (리스트나 딕셔너리가 아닙니다).")

    print(f"✅ DPO 데이터 {len(data_dpo)}개 로드 완료.")

except json.JSONDecodeError:
    # 혹시라도 JSONL(줄단위) 형식일 경우를 대비한 예외처리
    print("⚠️ 일반 JSON 로드 실패, JSONL 방식으로 재시도합니다...")
    data_dpo = []
    with open(file_path_dpo, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                try:
                    data_dpo.append(json.loads(line))
                except:
                    continue
    print(f"✅ (JSONL 방식) DPO 데이터 {len(data_dpo)}개 로드 완료.")

# 이후 코드는 동일하게 진행
dataset_dpo = Dataset.from_list(data_dpo)
# ... 포맷팅 함수 적용 및 Trainer 실행 ...

processed_dataset_dpo = dataset_dpo.map(format_dpo_func)

print("\n[DPO 데이터 예시 확인]")
print(f"Prompt: {processed_dataset_dpo[0]['prompt']}")
print(f"Chosen: {processed_dataset_dpo[0]['chosen']}")
print(f"Rejected: {processed_dataset_dpo[0]['rejected']}")


# 5. DPO Config 설정
dpo_config = DPOConfig(
    output_dir = "dpo_results",     # 결과 저장 경로
    beta = 0.1,                     # DPO 핵심 파라미터 (보통 0.1 사용)
    dataset_num_proc = 2,
    max_length = 1024,              # Prompt + Response 최대 길이
    max_prompt_length = 512,        # Prompt 최대 길이

    # 학습 파라미터 (SFT보다 학습률을 낮게 잡는 것이 일반적)
    learning_rate = 5e-6,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    num_train_epochs = 1,           # DPO는 과적합이 빠르므로 1 epoch 추천
    logging_steps = 1,

    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 42,
)

# 6. DPOTrainer 실행
trainer = DPOTrainer(
    model = model,
    ref_model = None, # Unsloth/TRL이 알아서 처리 (메모리 절약)
    tokenizer = tokenizer,
    train_dataset = processed_dataset_dpo,
    args = dpo_config,
)

print("\n🚀 DPO 학습 시작...")
trainer.train()

# 7. 모델 저장
trainer.save_model("dpo_final_model")
print("✅ DPO 학습 완료 및 저장됨: dpo_final_model")

==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Already have LoRA adapters! We shall skip this step.


✅ SFT 모델 로드 성공: lora_sft_output
✅ DPO 데이터 600개 로드 완료.


Map:   0%|          | 0/600 [00:00<?, ? examples/s]


[DPO 데이터 예시 확인]
Prompt: <|im_start|>system
You are an expert Korean translator.
You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.
Do not translate word-for-word; preserve meaning and tone in Korean.<|im_end|>
<|im_start|>user
Instruction: Don't translate it in Korean, but translate it according to Korean culture
Input: I actually saw him at the mall yesterday, no cap.<|im_end|>
<|im_start|>assistant

Chosen: 나 어제 쇼핑몰에서 걔 진짜로 봤다니까, 구라 아님.<|im_end|>
Rejected: 나 사실 어제 쇼핑몰에서 그를 봤어, 모자 없음.<|im_end|>


...

The model is already on multiple devices. Skipping the move to device specified in `args`.



🚀 DPO 학습 시작...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 1 | Total steps = 75
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.434400,2.728185,1.936413,0.875000,0.791772,-86.765396,-82.320732,-1.270341,-1.486429,0,0,0
2,0.261400,3.487968,1.949863,1.000000,1.538106,-97.045471,-65.984970,-1.209784,-1.429120,No Log,No Log,No Log
3,0.326500,2.852363,1.686264,1.000000,1.166098,-88.520004,-72.731567,-1.381505,-1.387306,No Log,No Log,No Log
4,0.310800,2.729288,1.521369,1.000000,1.207920,-92.026573,-77.933151,-1.183000,-1.413471,No Log,No Log,No Log
5,0.257800,2.957745,1.482358,1.000000,1.475386,-106.188332,-83.879608,-1.102493,-1.069365,No Log,No Log,No Log
6,0.230900,3.421199,1.939394,1.000000,1.481804,-90.901840,-76.102448,-1.283231,-1.376236,No Log,No Log,No Log
7,0.211000,3.466658,1.506558,1.000000,1.960099,-99.616898,-82.812180,-1.203192,-1.617713,No Log,No Log,No Log
8,0.216500,3.247753,1.221295,1.000000,2.026458,-103.861458,-89.453156,-1.327096,-1.515755,No Log,No Log,No Log
9,0.280000,2.697292,1.452520,1.000000,1.244772,-85.742477,-69.507698,-1.207774,-1.197803,No Log,No Log,No Log
10,0.271100,3.030875,1.318793,0.875000,1.712082,-106.043808,-83.521744,-1.316678,-1.397064,No Log,No Log,No Log


✅ DPO 학습 완료 및 저장됨: dpo_final_model


In [13]:
import torch
import pandas as pd
from unsloth import FastLanguageModel

# ------------------------------------------------------------------
# 1. 시드(Seed) 설정 (재현성 확보)
# ------------------------------------------------------------------
SEED = 42  # 원하는 숫자로 변경 가능

# ------------------------------------------------------------------
# 2. DPO 학습된 모델 로드 (Unsloth)
# ------------------------------------------------------------------
model_path = "dpo_final_model"  # DPO 학습 완료된 모델 경로

print(f"🔄 모델 로드 중: {model_path} ...")
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_path,
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    # 추론 속도 최적화
    FastLanguageModel.for_inference(model)
    print("✅ 모델 로드 및 추론 최적화 완료!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    raise e

# ------------------------------------------------------------------
# 3. 데이터 로드 및 샘플링 (HyperClovaX 데이터 포맷 동일)
# ------------------------------------------------------------------
file_path = "test_data.csv" # 업로드된 CSV 파일 경로

try:
    df = pd.read_csv(file_path)

    # [중요] HyperClovaX 코드와 동일한 컬럼명 매핑
    test_df = df.rename(columns={
        "영어 원문 (Source Text)": "input",
        "초월 번역": "output"
    })

    # 샘플링 개수 설정
    sample_size = 25

    # 데이터 개수 확인 후 안전하게 샘플링
    real_size = min(len(test_df), sample_size)

    # ✅ 여기서 random_state를 사용하여 시드를 고정합니다.
    sampled_df = test_df.sample(n=real_size, random_state=SEED)

    print(f"📊 테스트 데이터: 총 {len(sampled_df)}개 샘플링 완료 (Seed={SEED}).\n")

except FileNotFoundError:
    print(f"❌ '{file_path}' 파일을 찾을 수 없습니다. 파일을 업로드해주세요.")
    sampled_df = pd.DataFrame()

# ------------------------------------------------------------------
# 4. 프롬프트 및 생성 함수 설정
# ------------------------------------------------------------------
SYSTEM_PROMPT = (
    "You are an expert Korean translator.\n"
    "You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.\n"
    "Do not translate word-for-word; preserve meaning and tone in Korean."
)
INSTRUCTION = "Don't translate it in Korean, but translate it according to Korean culture"

def generate_translation(text):
    # Unsloth/Qwen 포맷 적용
    prompt = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n"
        f"Instruction: {INSTRUCTION}\n"
        f"Input: {text}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 결과 파싱
    try:
        if "assistant\n" in decoded:
            result = decoded.split("assistant\n")[-1].strip()
        else:
            result = decoded
    except:
        result = decoded

    return result

import json  # JSON 저장을 위해 최상단 혹은 해당 섹션 위에 import 필요

# ------------------------------------------------------------------
# 5. 실행, 결과 비교 출력 및 JSON 저장
# ------------------------------------------------------------------
output_json_path = "dpo_test_results.json"  # 저장할 파일명
results_list = []  # 결과를 저장할 리스트 초기화

if not sampled_df.empty:
    print(f"{'='*20} [DPO 모델 테스트 결과 비교 (Seed: {SEED})] {'='*20}\n")

    for idx, row in sampled_df.iterrows():
        input_text = row['input']
        ref_text = row['output']  # 정답 데이터

        # 모델 예측
        model_out = generate_translation(input_text)

        # 1. 콘솔 출력
        print(f"[Sample {idx}]")
        print(f"🇺🇸 원문      : {input_text}")
        print(f"🇰🇷 정답(Ref) : {ref_text}")
        print(f"🔥 DPO Model : {model_out}")
        print("-" * 60)

        # 2. 결과 데이터 구조화 (JSON 저장용)
        result_data = {
            "id": idx,
            "input": input_text,
            "reference": ref_text,
            "prediction": model_out
        }
        results_list.append(result_data)

    # 3. JSON 파일로 저장
    try:
        with open(output_json_path, "w", encoding="utf-8") as f:
            # ensure_ascii=False를 해야 한글이 유니코드(\uXXXX)가 아닌 글자 그대로 저장됨
            json.dump(results_list, f, ensure_ascii=False, indent=4)

        print(f"\n💾 결과가 성공적으로 저장되었습니다: {output_json_path}")
        print(f"📊 총 저장된 샘플 수: {len(results_list)}")

    except Exception as e:
        print(f"❌ JSON 저장 중 오류 발생: {e}")

else:
    print("⚠️ 테스트할 데이터가 없습니다.")

🔄 모델 로드 중: dpo_final_model ...
==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ 모델 로드 및 추론 최적화 완료!
📊 테스트 데이터: 총 25개 샘플링 완료 (Seed=42).

==================== [DPO 모델 테스트 결과 비교 (Seed: 42)] ====================

[Sample 62]
🇺🇸 원문      : There is a tide in the affairs of men... leads on to fortune.
🇰🇷 정답(Ref) : 물들어 올때 노저어야 한다. 
🔥 DPO Model : <think>
다리가 난 그의 일치가 나서(이 상황이 진짜 진짜) 보이지 않으면 진짜 빼내는 거야(이 상황은 끼어내는 거야).
------------------------------------------------------------
[Sample 40]
🇺🇸 원문      : They expected him to crack under pressure, but he stayed cool as a cucumber.
🇰🇷 정답(Ref) : 다 긴장했는데 걔 혼자

In [14]:
import json
import pandas as pd
import sacrebleu

# ------------------------------------------------------------------
# 1. 파일 로드 함수
# ------------------------------------------------------------------
def load_results(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {json_path}")
        return []

# 경로 설정 (앞선 코드에서 저장한 파일명)
sft_path = "sft_test_results.json"
dpo_path = "dpo_test_results.json"

print("📂 데이터 로드 중...")
sft_data = load_results(sft_path)
dpo_data = load_results(dpo_path)

# ------------------------------------------------------------------
# 2. 데이터 정렬 및 리스트 추출
# ------------------------------------------------------------------
# 데이터가 같은 순서인지 확인하기 위해 ID 기준으로 딕셔너리 변환
sft_dict = {item['id']: item for item in sft_data}
dpo_dict = {item['id']: item for item in dpo_data}

# 공통된 ID만 추출 (혹시 모를 데이터 불일치 방지)
common_ids = sorted(list(set(sft_dict.keys()) & set(dpo_dict.keys())))

refs = []       # 정답 (Reference)
sys_sft = []    # SFT 예측값
sys_dpo = []    # DPO 예측값
inputs = []     # 원문 (비교용)

for i in common_ids:
    refs.append(sft_dict[i]['reference']) # 정답은 둘 다 동일하므로 SFT에서 가져옴
    sys_sft.append(sft_dict[i].get('prediction_sft', sft_dict[i].get('prediction')))
    sys_dpo.append(dpo_dict[i].get('prediction_dpo', dpo_dict[i].get('prediction')))
    inputs.append(sft_dict[i]['input'])

print(f"✅ 총 {len(common_ids)}개의 공통 샘플에 대해 평가를 진행합니다.\n")

# ------------------------------------------------------------------
# 3. BLEU 스코어 계산 (SacreBLEU)
# ------------------------------------------------------------------
# 한국어는 'ko-mecab' 토크나이저가 가장 정확하지만,
# 설치 문제 방지를 위해 일반적인 '13a' 혹은 'spm'을 주로 사용합니다.
# 여기서는 기본값(13a)을 사용합니다.

bleu_sft = sacrebleu.corpus_bleu(sys_sft, [refs])
bleu_dpo = sacrebleu.corpus_bleu(sys_dpo, [refs])

print(f"{'='*10} 📊 BLEU Score 결과 {'='*10}")
print(f"🔹 SFT Model BLEU : {bleu_sft.score:.2f}")
print(f"🔥 DPO Model BLEU : {bleu_dpo.score:.2f}")
print(f"{'='*40}\n")

# 점수 차이 분석
diff = bleu_dpo.score - bleu_sft.score
if diff > 0:
    print(f"🚀 DPO 모델이 SFT보다 +{diff:.2f} 점 더 높습니다!")
else:
    print(f"📉 DPO 모델 점수가 SFT보다 {abs(diff):.2f} 점 낮습니다. (과적합 혹은 데이터 확인 필요)")

# ------------------------------------------------------------------
# 4. 결과 상세 비교 (Excel/CSV 저장용 데이터프레임)
# ------------------------------------------------------------------
df_compare = pd.DataFrame({
    'Input': inputs,
    'Reference': refs,
    'SFT_Pred': sys_sft,
    'DPO_Pred': sys_dpo
})

# 화면에 랜덤 3개 샘플 출력
print("\n[👀 결과 비교 예시 (Random 3)]")
print("-" * 60)
for idx, row in df_compare.sample(n=min(3, len(df_compare)), random_state=42).iterrows():
    print(f"🇺🇸 원문: {row['Input']}")
    print(f"🇰🇷 정답: {row['Reference']}")
    print(f"🔹 SFT : {row['SFT_Pred']}")
    print(f"🔥 DPO : {row['DPO_Pred']}")
    print("-" * 60)

# (선택) 결과를 엑셀이나 CSV로 저장하고 싶다면 주석 해제
# df_compare.to_csv("model_comparison.csv", index=False, encoding="utf-8-sig")
# print("\n💾 전체 비교 결과가 'model_comparison.csv'로 저장되었습니다.")

📂 데이터 로드 중...
✅ 총 25개의 공통 샘플에 대해 평가를 진행합니다.

========== 📊 BLEU Score 결과 ==========
🔹 SFT Model BLEU : 0.56
🔥 DPO Model BLEU : 0.34

📉 DPO 모델 점수가 SFT보다 0.23 점 낮습니다. (과적합 혹은 데이터 확인 필요)

[👀 결과 비교 예시 (Random 3)]
------------------------------------------------------------
🇺🇸 원문: You can try convincing her, but she’s stubborn as a mule—good luck with that.
🇰🇷 정답: 설득하려고 해도 소용 없어. 그냥 고집불통이야
🔹 SFT : 
🔥 DPO : <think>
네, 한국어에 맞춰서 말 해. 거 죽을 거야, 그건 말 해. 어짜리, 나 봐.
------------------------------------------------------------
🇺🇸 원문: ...from hell’s heart I stab at thee; for hate’s sake...
🇰🇷 정답: 지옥 같은 심장으로 마지막 창을 꽂겠다—증오를 다해.
🔹 SFT : <think>
네, Korean culture에 맞춰서 말하지만 Korean어로 해서 말.
</think>

네, Korean 문화에 맞춰서 해보지만 Korean어로 해.
🔥 DPO : <think>
네, Korean 문법에 맞게 해줘. 이 문장은 Korean 대중문화 표현이면서야.
------------------------------------------------------------
🇺🇸 원문: The committee, whose investigation opened a can of worms regarding the project's funding, is expected to release its full report.
🇰🇷 정답: 그 프로젝트의 자금 

In [16]:
import torch
import pandas as pd
import json
from unsloth import FastLanguageModel
from tqdm import tqdm  # 진행률 표시바

# ==========================================
# 👇 실행할 때마다 이 부분만 수정하세요!
# ==========================================
# [Case 1: SFT 모델 실행 시]
model_dir = "lora_sft_output"
save_file = "sft_full_results.json"

# [Case 2: DPO 모델 실행 시]
# model_dir = "dpo_final_model"
# save_file = "dpo_full_results.json"
# ==========================================

# 1. 모델 로드
print(f"🔄 모델 로드 중: {model_dir} ...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_dir,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# 2. 전체 데이터 로드 (샘플링 X)
csv_path = "test_data.csv"
try:
    df = pd.read_csv(csv_path)
    # 컬럼명 매핑
    df = df.rename(columns={
        "영어 원문 (Source Text)": "input",
        "초월 번역": "output"
    })
    print(f"📊 전체 데이터 {len(df)}개를 로드했습니다.")
except Exception as e:
    print("❌ CSV 파일을 찾을 수 없습니다.")
    raise e

# 3. 프롬프트 설정
SYSTEM_PROMPT = (
    "You are an expert Korean translator.\n"
    "You translate English sentences into natural Korean, adapting to Korean internet and youth culture expressions.\n"
    "Do not translate word-for-word; preserve meaning and tone in Korean."
)
INSTRUCTION = "Don't translate it in Korean, but translate it according to Korean culture"

def generate_translation(text):
    prompt = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n"
        f"Instruction: {INSTRUCTION}\n"
        f"Input: {text}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs, max_new_tokens=128, temperature=0.7, top_p=0.9,
        use_cache=True, pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("assistant\n")[-1].strip() if "assistant\n" in decoded else decoded

# 4. 전체 데이터 추론 실행
results = []
print(f"🚀 전체 데이터 추론 시작 (저장 파일명: {save_file})")

# tqdm을 사용하여 진행률 표시
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Translating"):
    input_text = row['input']
    ref_text = row['output']

    # 번역 생성
    pred = generate_translation(input_text)

    results.append({
        "id": idx,
        "input": input_text,
        "reference": ref_text,
        "prediction": pred
    })

# 5. 결과 저장
with open(save_file, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"\n💾 저장 완료! 파일명: {save_file}")

🔄 모델 로드 중: lora_sft_output ...
==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
📊 전체 데이터 99개를 로드했습니다.
🚀 전체 데이터 추론 시작 (저장 파일명: sft_full_results.json)


Translating: 100%|██████████| 99/99 [03:05<00:00,  1.87s/it]


💾 저장 완료! 파일명: sft_full_results.json


In [17]:
import json
import pandas as pd
import sacrebleu

# 1. 파일 경로 설정 (전체 데이터 파일)
sft_path = "sft_full_results.json"
dpo_path = "dpo_full_results.json"

def load_results(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {json_path}")
        return []

print("📂 전체 데이터 결과 로드 중...")
sft_data = load_results(sft_path)
dpo_data = load_results(dpo_path)

if not sft_data or not dpo_data:
    print("⚠️ 결과 파일이 없어 평가를 중단합니다. 1단계(추론)를 먼저 완료해주세요.")
else:
    # 2. 데이터 매칭 (ID 기준)
    sft_dict = {item['id']: item for item in sft_data}
    dpo_dict = {item['id']: item for item in dpo_data}

    common_ids = sorted(list(set(sft_dict.keys()) & set(dpo_dict.keys())))

    refs = []
    sys_sft = []
    sys_dpo = []
    inputs = []

    for i in common_ids:
        refs.append(sft_dict[i]['reference'])
        # prediction 키 이름이 다를 수 있으므로 get으로 안전하게 호출
        sys_sft.append(sft_dict[i].get('prediction', sft_dict[i].get('prediction_sft')))
        sys_dpo.append(dpo_dict[i].get('prediction', dpo_dict[i].get('prediction_dpo')))
        inputs.append(sft_dict[i]['input'])

    print(f"✅ 총 {len(common_ids)}개의 전체 샘플에 대해 평가합니다.\n")

    # 3. BLEU 스코어 계산
    bleu_sft = sacrebleu.corpus_bleu(sys_sft, [refs])
    bleu_dpo = sacrebleu.corpus_bleu(sys_dpo, [refs])

    print(f"{'='*10} 📊 전체 데이터 BLEU Score {'='*10}")
    print(f"🔹 SFT Model : {bleu_sft.score:.2f}")
    print(f"🔥 DPO Model : {bleu_dpo.score:.2f}")
    print(f"{'='*40}\n")

    diff = bleu_dpo.score - bleu_sft.score
    if diff > 0:
        print(f"🚀 DPO가 전체적으로 +{diff:.2f} 점 더 우수합니다!")
    else:
        print(f"📉 DPO가 SFT보다 {abs(diff):.2f} 점 낮습니다.")

    # 4. 전체 결과 CSV 저장 (분석용)
    df_compare = pd.DataFrame({
        'Input': inputs,
        'Reference': refs,
        'SFT_Pred': sys_sft,
        'DPO_Pred': sys_dpo
    })

    csv_save_name = "final_full_comparison.csv"
    df_compare.to_csv(csv_save_name, index=False, encoding="utf-8-sig")
    print(f"\n💾 전체 비교 결과가 '{csv_save_name}' 파일로 저장되었습니다.")

📂 전체 데이터 결과 로드 중...
✅ 총 99개의 전체 샘플에 대해 평가합니다.

========== 📊 전체 데이터 BLEU Score ==========
🔹 SFT Model : 0.28
🔥 DPO Model : 0.13

📉 DPO가 SFT보다 0.15 점 낮습니다.

💾 전체 비교 결과가 'final_full_comparison.csv' 파일로 저장되었습니다.
